In [73]:
import os, glob
os.getcwd()
import pandas as pd
brown = pd.read_csv('archive-2/brown.csv')
print(brown.shape)
brown.head()

(57340, 7)


,filename,para_id,sent_id,raw_text,tokenized_text,tokenized_pos,label
0,cd05,0,0,"Furthermore/rb ,/, as/cs an/at encouragement/n...","Furthermore , as an encouragement to revisioni...","rb , cs at nn in nn nn , pps rb bez jj to vb c...",religion
1,cd05,0,1,The/at Unitarian/jj clergy/nns were/bed an/at ...,The Unitarian clergy were an exclusive club of...,at jj nns bed at jj nn in vbn nns -- cs at nn ...,religion
2,cd05,0,2,"Ezra/np Stiles/np Gannett/np ,/, an/at honorab...","Ezra Stiles Gannett , an honorable representat...","np np np , at jj nn in at nn , vbd ppl rb in a...",religion
3,cd05,0,3,"Even/rb so/rb ,/, Gannett/np judiciously/rb ar...","Even so , Gannett judiciously argued , the Ass...","rb rb , np rb vbd , at nn-tl md rb vb cs np ``...",religion
4,cd05,0,4,We/ppss today/nr are/ber not/* entitled/vbn to...,We today are not entitled to excoriate honest ...,ppss nr ber * vbn to vb jj nns wps vbd np to b...,religion


In [74]:
words=0
list_word=[]
for i in brown['tokenized_text']:
    word=i.split()
    for w in word:
        words+=1
        list_word.append(w)
print("le somme des mots dans l'ensemble:")
len(set(list_word))
words

le somme des mots dans l'ensemble:


1161169

In [75]:
result = zip(brown['label'], brown['raw_text'])

r=list(result)
dic={}
for i in r:
    dic[i[0]]=i[1]


data = {'Catégorie':dic.keys(), 'Nombre':([len(contenu) for k,contenu in dic.items()])}
df = pd.DataFrame(data)

df

,Catégorie,Nombre
0,religion,354
1,lore,224
2,learned,263
3,mystery,25
4,adventure,89
5,government,223
6,editorial,314
7,humor,250
8,hobbies,429
9,belles_lettres,499


In [76]:
print("longueur de verteur : ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N))
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(brown['raw_text']).getnnz())

longueur de verteur : 
ngram_range (1, 1) :
1425024
ngram_range (1, 2) :
3257472
ngram_range (1, 3) :
5134617


In [77]:
print("longueur de verteur sans stopwords: ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False,stop_words='english')
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(brown['raw_text']).getnnz())

longueur de verteur sans stopwords: 
ngram_range (1, 1) :
991438
ngram_range (1, 2) :
2227475
ngram_range (1, 3) :
3435181


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import time


In [93]:
Resultat=[]

In [78]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
liste_classifieurs= [
    ["Perceptron", Perceptron(eta0=0.1, random_state=0)],
    ["MultinomialNB", MultinomialNB()],
    ["Logistic Regression", LogisticRegression()],
    ["linear_svc", LinearSVC()],
    ["Random Forest",RandomForestClassifier(n_estimators=250,max_depth=4, random_state=0)],
    ["Decision Tree", DecisionTreeClassifier(criterion='gini',max_depth=4,splitter='random',min_samples_split=5)],
    ["Perceptron multicouche",MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(30,20), random_state=1)]

]
## Pour éviter les warnings (merci julien):
import warnings 
warnings.filterwarnings("ignore")

In [94]:
import json
import os
chemin_expes = "Pos_original.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


Classification utilisant les textes avec POS:

In [80]:
en_minuscules,enlever_stopwords  = False, False
from sklearn.metrics import classification_report 

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, False, False] 
 0.4644808743169399 
                  precision    recall  f1-score   support

        learned       0.43      0.47      0.45      1449
        mystery       0.39      0.57      0.46      2121
     government       0.29      0.32      0.30       904
        hobbies       0.44      0.35      0.39      1268
           news       0.60      0.54      0.57       890
        fiction       0.52      0.58      0.55      1281
        reviews       0.35      0.20      0.26       314
           lore       0.70      0.62      0.66      2288
      editorial       0.50      0.31      0.39      1422
 belles_lettres       0.44      0.43      0.44      1186
      adventure       0.49      0.51      0.50      1387
        romance       0.32      0.42      0.37       515
       religion       0.46      0.31      0.37       550
science_fiction       0.42      0.37      0.40      1361
          humor       0.27      0.35   

In [95]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.5413905359841878, "['linear_svc', 1, 2, False, False]"]
[0.535344727357284, "['linear_svc', 1, 1, False, False]"]
[0.5286013254272759, "['linear_svc', 1, 3, False, False]"]
[0.4856412045111034, "['Perceptron', 1, 2, False, False]"]
[0.4793047320079061, "['Perceptron', 1, 3, False, False]"]
[0.46918962911289386, "['Logistic Regression', 1, 1, False, False]"]
[0.4644808743169399, "['Perceptron', 1, 1, False, False]"]
[0.4629694221602139, "['Logistic Regression', 1, 2, False, False]"]
[0.432682246250436, "['Logistic Regression', 1, 3, False, False]"]
[0.41721892803162425, "['Perceptron multicouche', 1, 1, False, False]"]
[0.3874549470991745, "['Perceptron multicouche', 1, 3, False, False]"]
[0.37449133821648645, "['Perceptron multicouche', 1, 2, False, False]"]
[0.3057202650854552, "['MultinomialNB', 1, 1, False, False]"]
[0.28618765259853507, "['MultinomialNB', 1, 2, False, False]"]
[0.26898035112196256, "['MultinomialNB', 1, 3, False, False]"]
[0.1926520172073015, "['Decision Tree', 

In [115]:
en_minuscules,enlever_stopwords  = False, False


for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        clf = algo.fit(X_train, y_train)
        score = clf.score(X_train, y_train)
        
        print(nom,score)

---------------
Ngram_range : (1, 1)
Perceptron 0.8964073944890129
MultinomialNB 0.4001694155164682
Logistic Regression 0.6609945687378544
linear_svc 0.9301659275499526
Random Forest 0.14689321839653197
Decision Tree 0.17995415815436744
Perceptron multicouche 0.7061388210673177
---------------
Ngram_range : (1, 2)
Perceptron 0.9825601674223927
MultinomialNB 0.4022871094723205
Logistic Regression 0.7295580248143904
linear_svc 0.9857491653794409
Random Forest 0.1445512980218247
Decision Tree 0.18189745378444366
Perceptron multicouche 0.7972744033085853
---------------
Ngram_range : (1, 3)
Perceptron 0.9878668593352933
MultinomialNB 0.42503363396282823
Logistic Regression 0.7654093377846429
linear_svc 0.9904080920823161
Random Forest 0.1433305097413922
Decision Tree 0.17208131944790472


KeyboardInterrupt: 

In [97]:

chemin_expes = "Pos_lower.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [83]:
en_minuscules,enlever_stopwords  = True, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N))
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)



---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, False, True] 
 0.45907452621788164 
                  precision    recall  f1-score   support

        learned       0.48      0.39      0.43      1449
        mystery       0.48      0.45      0.47      2121
     government       0.27      0.39      0.32       904
        hobbies       0.36      0.43      0.39      1268
           news       0.52      0.60      0.56       890
        fiction       0.58      0.53      0.56      1281
        reviews       0.29      0.22      0.25       314
           lore       0.65      0.64      0.65      2288
      editorial       0.49      0.29      0.36      1422
 belles_lettres       0.44      0.42      0.43      1186
      adventure       0.50      0.51      0.50      1387
        romance       0.30      0.48      0.37       515
       religion       0.41      0.30      0.35       550
science_fiction       0.39      0.45      0.42      1361
          humor       0.24      0.32   

In [98]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.5412161376584118, "['linear_svc', 1, 2, False, True]"]
[0.5331356818974538, "['linear_svc', 1, 1, False, True]"]
[0.5297639809324497, "['linear_svc', 1, 3, False, True]"]
[0.48378095570282525, "['Perceptron', 1, 2, False, True]"]
[0.48069991861411465, "['Perceptron', 1, 3, False, True]"]
[0.46750377863039183, "['Logistic Regression', 1, 2, False, True]"]
[0.4658760609231485, "['Logistic Regression', 1, 1, False, True]"]
[0.45907452621788164, "['Perceptron', 1, 1, False, True]"]
[0.4296593419369841, "['Logistic Regression', 1, 3, False, True]"]
[0.4180327868852459, "['Perceptron multicouche', 1, 1, False, True]"]
[0.4040227880479014, "['Perceptron multicouche', 1, 2, False, True]"]
[0.3730380188350192, "['Perceptron multicouche', 1, 3, False, True]"]
[0.3107196837577026, "['MultinomialNB', 1, 1, False, True]"]
[0.28874549470991745, "['MultinomialNB', 1, 2, False, True]"]
[0.270026741076619, "['MultinomialNB', 1, 3, False, True]"]
[0.19137309615161027, "['Decision Tree', 1, 3, False, 

In [ ]:
en_minuscules,enlever_stopwords  = True, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N))
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
       
        clf = algo.fit(X_train, y_train)
        score = clf.score(X_train, y_train)

        print(nom,score)

In [99]:
chemin_expes = "Pos_lower_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [87]:
en_minuscules,enlever_stopwords  = True, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),max_df = 0.7)
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            #print(calcul_time)

---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, True, True] 
 0.4611091733519358 
                  precision    recall  f1-score   support

        learned       0.42      0.46      0.44      1449
        mystery       0.38      0.57      0.46      2121
     government       0.34      0.31      0.33       904
        hobbies       0.40      0.35      0.37      1268
           news       0.66      0.49      0.56       890
        fiction       0.50      0.59      0.54      1281
        reviews       0.35      0.19      0.25       314
           lore       0.67      0.63      0.65      2288
      editorial       0.49      0.35      0.41      1422
 belles_lettres       0.38      0.44      0.41      1186
      adventure       0.53      0.49      0.51      1387
        romance       0.39      0.34      0.36       515
       religion       0.50      0.27      0.35       550
science_fiction       0.39      0.40      0.39      1361
          humor       0.33      0.27     

In [100]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.5411580048831531, "['linear_svc', 1, 2, True, True]"]
[0.5330775491221951, "['linear_svc', 1, 1, True, True]"]
[0.529880246482967, "['linear_svc', 1, 3, True, True]"]
[0.4842460179048948, "['Perceptron', 1, 2, True, True]"]
[0.47552610161609116, "['Perceptron', 1, 3, True, True]"]
[0.47058481571910243, "['Logistic Regression', 1, 1, True, True]"]
[0.46558539704685503, "['Logistic Regression', 1, 2, True, True]"]
[0.4611091733519358, "['Perceptron', 1, 1, True, True]"]
[0.43680967329380305, "['Logistic Regression', 1, 3, True, True]"]
[0.42285780723171723, "['Perceptron multicouche', 1, 1, True, True]"]
[0.3768166492268341, "['Perceptron multicouche', 1, 3, True, True]"]
[0.35280781304499476, "['Perceptron multicouche', 1, 2, True, True]"]
[0.3249040809208232, "['MultinomialNB', 1, 1, True, True]"]
[0.2960120916172538, "['MultinomialNB', 1, 2, True, True]"]
[0.27589815137774676, "['MultinomialNB', 1, 3, True, True]"]
[0.18149052435763283, "['Decision Tree', 1, 3, True, True]"]
[0.177

In [101]:
chemin_expes = "Pos_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [90]:
en_minuscules,enlever_stopwords  = False, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),max_df = 0.7,lowercase = False)
    X = V.fit_transform(brown['raw_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            score = dic_expes[expe]
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            
            print(report)
            #print(calcul_time)

---------------
Ngram_range : (1, 1)
  Déjà vu
['Perceptron', 1, 1, True, False] 
 0.46407394489012904 
                  precision    recall  f1-score   support

        learned       0.50      0.40      0.44      1449
        mystery       0.38      0.54      0.45      2121
     government       0.32      0.29      0.30       904
        hobbies       0.41      0.39      0.40      1268
           news       0.62      0.57      0.59       890
        fiction       0.54      0.55      0.55      1281
        reviews       0.19      0.26      0.22       314
           lore       0.70      0.64      0.67      2288
      editorial       0.50      0.34      0.40      1422
 belles_lettres       0.43      0.44      0.43      1186
      adventure       0.45      0.53      0.49      1387
        romance       0.39      0.38      0.38       515
       religion       0.46      0.31      0.37       550
science_fiction       0.42      0.40      0.41      1361
          humor       0.24      0.35   

In [102]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Resultat.append(res)

[0.5415068015347053, "['linear_svc', 1, 2, True, False]"]
[0.5340076735263342, "['linear_svc', 1, 1, True, False]"]
[0.5282525287757237, "['linear_svc', 1, 3, True, False]"]
[0.48703639111731195, "['Perceptron', 1, 2, True, False]"]
[0.48133937914196023, "['Perceptron', 1, 3, True, False]"]
[0.4688408324613417, "['Logistic Regression', 1, 1, True, False]"]
[0.46424834321590513, "['Logistic Regression', 1, 2, True, False]"]
[0.46407394489012904, "['Perceptron', 1, 1, True, False]"]
[0.4321590512731078, "['Logistic Regression', 1, 3, True, False]"]
[0.43169398907103823, "['Perceptron multicouche', 1, 1, True, False]"]
[0.3856528310661551, "['Perceptron multicouche', 1, 2, True, False]"]
[0.36356237646785255, "['Perceptron multicouche', 1, 3, True, False]"]
[0.3203115916753866, "['MultinomialNB', 1, 1, True, False]"]
[0.29258225787699105, "['MultinomialNB', 1, 2, True, False]"]
[0.2736891059179165, "['MultinomialNB', 1, 3, True, False]"]
[0.18462969422160214, "['Decision Tree', 1, 2, True

In [103]:
for res in sorted(Resultat,reverse=True):
    print(res)

[0.5415068015347053, "['linear_svc', 1, 2, True, False]"]
[0.5413905359841878, "['linear_svc', 1, 2, False, False]"]
[0.5412161376584118, "['linear_svc', 1, 2, False, True]"]
[0.5411580048831531, "['linear_svc', 1, 2, True, True]"]
[0.535344727357284, "['linear_svc', 1, 1, False, False]"]
[0.5340076735263342, "['linear_svc', 1, 1, True, False]"]
[0.5331356818974538, "['linear_svc', 1, 1, False, True]"]
[0.5330775491221951, "['linear_svc', 1, 1, True, True]"]
[0.529880246482967, "['linear_svc', 1, 3, True, True]"]
[0.5297639809324497, "['linear_svc', 1, 3, False, True]"]
[0.5286013254272759, "['linear_svc', 1, 3, False, False]"]
[0.5282525287757237, "['linear_svc', 1, 3, True, False]"]
[0.48703639111731195, "['Perceptron', 1, 2, True, False]"]
[0.4856412045111034, "['Perceptron', 1, 2, False, False]"]
[0.4842460179048948, "['Perceptron', 1, 2, True, True]"]
[0.48378095570282525, "['Perceptron', 1, 2, False, True]"]
[0.48133937914196023, "['Perceptron', 1, 3, True, False]"]
[0.4806999186

Clacification des textes sans POS

In [34]:
print("longueur de verteur : ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False)
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(brown['tokenized_text']).getnnz())


longueur de verteur : 
ngram_range (1, 1) :
889937
ngram_range (1, 2) :
1812617
ngram_range (1, 3) :
2686375


In [35]:
print("longueur de verteur sans stopwords: ")
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase = False,stop_words='english')
    print("ngram_range",(min_N, max_N),":")
    print(V.fit_transform(brown['tokenized_text']).getnnz())

longueur de verteur sans stopwords: 
ngram_range (1, 1) :
512062
ngram_range (1, 2) :
978828
ngram_range (1, 3) :
1391398


In [104]:
Result=[]

In [105]:
chemin_expes = "Brown_original.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [57]:
en_minuscules,enlever_stopwords  = False, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N))
    X = V.fit_transform(brown['tokenized_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.4488
                 precision    recall  f1-score   support

        learned       0.40      0.48      0.43      1449
        mystery       0.38      0.55      0.45      2121
     government       0.36      0.28      0.32       904
        hobbies       0.37      0.37      0.37      1268
           news       0.51      0.54      0.52       890
        fiction       0.53      0.52      0.52      1281
        reviews       0.32      0.22      0.26       314
           lore       0.67      0.62      0.64      2288
      editorial       0.46      0.30      0.37      1422
 belles_lettres       0.46      0.38      0.42      1186
      adventure       0.50      0.47      0.48      1387
        romance       0.34      0.38      0.36       515
       religion       0.43      0.31      0.36       550
science_fiction       0.40      0.39      0.40      1361
          humor       0.25      0.30      0.27       266

       accuracy 

In [106]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Result.append(res)

[0.5264504127427043, "['linear_svc', 1, 1, False, False]"]
[0.5227299151261481, "['linear_svc', 1, 2, False, False]"]
[0.509010580165097, "['linear_svc', 1, 3, False, False]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, False, False]"]
[0.4658179281478898, "['Perceptron', 1, 2, False, False]"]
[0.45733054296012093, "['Perceptron', 1, 3, False, False]"]
[0.44884315777235206, "['Perceptron', 1, 1, False, False]"]
[0.44082083478665274, "['Perceptron multicouche', 1, 1, False, False]"]
[0.4337867689803511, "['Logistic Regression', 1, 2, False, False]"]
[0.395070340658063, "['Logistic Regression', 1, 3, False, False]"]
[0.38850133705383094, "['Perceptron multicouche', 1, 3, False, False]"]
[0.3855365655156377, "['Perceptron multicouche', 1, 2, False, False]"]
[0.35984187885129637, "['MultinomialNB', 1, 1, False, False]"]
[0.285257528194396, "['MultinomialNB', 1, 2, False, False]"]
[0.25863271712591557, "['MultinomialNB', 1, 3, False, False]"]
[0.16439948843157773, "['Decision Tree'

In [107]:
chemin_expes = "Brown_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [61]:
en_minuscules,enlever_stopwords  = False, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),max_df = 0.7)
    X = V.fit_transform(brown['tokenized_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.4488
                 precision    recall  f1-score   support

        learned       0.40      0.48      0.43      1449
        mystery       0.38      0.55      0.45      2121
     government       0.36      0.28      0.32       904
        hobbies       0.37      0.37      0.37      1268
           news       0.51      0.54      0.52       890
        fiction       0.53      0.52      0.52      1281
        reviews       0.32      0.22      0.26       314
           lore       0.67      0.62      0.64      2288
      editorial       0.46      0.30      0.37      1422
 belles_lettres       0.46      0.38      0.42      1186
      adventure       0.50      0.47      0.48      1387
        romance       0.34      0.38      0.36       515
       religion       0.43      0.31      0.36       550
science_fiction       0.40      0.39      0.40      1361
          humor       0.25      0.30      0.27       266

       accuracy 

In [108]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Result.append(res)

[0.5264504127427043, "['linear_svc', 1, 1, True, False]"]
[0.5227299151261481, "['linear_svc', 1, 2, True, False]"]
[0.509010580165097, "['linear_svc', 1, 3, True, False]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, True, False]"]
[0.4658179281478898, "['Perceptron', 1, 2, True, False]"]
[0.45733054296012093, "['Perceptron', 1, 3, True, False]"]
[0.44884315777235206, "['Perceptron', 1, 1, True, False]"]
[0.44082083478665274, "['Perceptron multicouche', 1, 1, True, False]"]
[0.4337867689803511, "['Logistic Regression', 1, 2, True, False]"]
[0.395070340658063, "['Logistic Regression', 1, 3, True, False]"]
[0.38850133705383094, "['Perceptron multicouche', 1, 3, True, False]"]
[0.3855365655156377, "['Perceptron multicouche', 1, 2, True, False]"]
[0.35984187885129637, "['MultinomialNB', 1, 1, True, False]"]
[0.285257528194396, "['MultinomialNB', 1, 2, True, False]"]
[0.25863271712591557, "['MultinomialNB', 1, 3, True, False]"]
[0.1627136379490757, "['Decision Tree', 1, 1, True, Fa

In [109]:
chemin_expes = "Brown_lower.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [65]:
en_minuscules,enlever_stopwords  = True, False

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=True)
    X = V.fit_transform(brown['tokenized_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            #print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.4488
('Temps_execution(secondes)', 0.6205136775970459)
  MultinomialNB classifier : 0.3598
('Temps_execution(secondes)', 0.36931395530700684)
  Logistic Regression classifier : 0.4794
('Temps_execution(secondes)', 7.0326879024505615)
  linear_svc classifier : 0.5265
('Temps_execution(secondes)', 1.5326461791992188)
  Random Forest classifier : 0.1395
('Temps_execution(secondes)', 6.034715890884399)
  Decision Tree classifier : 0.1456
('Temps_execution(secondes)', 0.9306049346923828)
  Perceptron multicouche classifier : 0.4408
('Temps_execution(secondes)', 26.77574896812439)
---------------
Ngram_range : (1, 2)
  Perceptron classifier : 0.4658
('Temps_execution(secondes)', 0.9400770664215088)
  MultinomialNB classifier : 0.2853
('Temps_execution(secondes)', 0.5081541538238525)
  Logistic Regression classifier : 0.4338
('Temps_execution(secondes)', 48.948009967803955)
  linear_svc classifier : 0.5227
('Temps_execution(seco

In [110]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Result.append(res)

[0.5264504127427043, "['linear_svc', 1, 1, False, True]"]
[0.5227299151261481, "['linear_svc', 1, 2, False, True]"]
[0.509010580165097, "['linear_svc', 1, 3, False, True]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, False, True]"]
[0.4658179281478898, "['Perceptron', 1, 2, False, True]"]
[0.45733054296012093, "['Perceptron', 1, 3, False, True]"]
[0.44884315777235206, "['Perceptron', 1, 1, False, True]"]
[0.44082083478665274, "['Perceptron multicouche', 1, 1, False, True]"]
[0.4337867689803511, "['Logistic Regression', 1, 2, False, True]"]
[0.395070340658063, "['Logistic Regression', 1, 3, False, True]"]
[0.38850133705383094, "['Perceptron multicouche', 1, 3, False, True]"]
[0.3855365655156377, "['Perceptron multicouche', 1, 2, False, True]"]
[0.35984187885129637, "['MultinomialNB', 1, 1, False, True]"]
[0.285257528194396, "['MultinomialNB', 1, 2, False, True]"]
[0.25863271712591557, "['MultinomialNB', 1, 3, False, True]"]
[0.1519009417509592, "['Decision Tree', 1, 2, False, T

In [111]:
chemin_expes = "Brown_lower_sw.json"

if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 21


In [69]:
en_minuscules,enlever_stopwords  = True, True

for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = TfidfVectorizer(ngram_range = (min_N, max_N),lowercase=True,max_df = 0.7)
    X = V.fit_transform(brown['tokenized_text'])
    y = brown['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print(expe, "\n",score[0],"\n",score[1],"\n",score[2])
            score = dic_expes[expe]
        else:
            T1=time.time()
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = [score]
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)
            nom_classes=set(brown['label'])
            report = classification_report(y_test,pred,target_names=nom_classes)
            T2=time.time()
            calcul_time = ("Temps_execution(secondes)",T2 - T1)
            dic_expes[expe].append(report)
            dic_expes[expe].append(calcul_time)
            print(report)
            print(calcul_time)

---------------
Ngram_range : (1, 1)
  Perceptron classifier : 0.4488
                 precision    recall  f1-score   support

        learned       0.40      0.48      0.43      1449
        mystery       0.38      0.55      0.45      2121
     government       0.36      0.28      0.32       904
        hobbies       0.37      0.37      0.37      1268
           news       0.51      0.54      0.52       890
        fiction       0.53      0.52      0.52      1281
        reviews       0.32      0.22      0.26       314
           lore       0.67      0.62      0.64      2288
      editorial       0.46      0.30      0.37      1422
 belles_lettres       0.46      0.38      0.42      1186
      adventure       0.50      0.47      0.48      1387
        romance       0.34      0.38      0.36       515
       religion       0.43      0.31      0.36       550
science_fiction       0.40      0.39      0.40      1361
          humor       0.25      0.30      0.27       266

       accuracy 

In [112]:
liste_resultats =[[score[0], nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats,reverse=True):
    print(res)
    Result.append(res)

[0.5264504127427043, "['linear_svc', 1, 1, True, True]"]
[0.5227299151261481, "['linear_svc', 1, 2, True, True]"]
[0.509010580165097, "['linear_svc', 1, 3, True, True]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, True, True]"]
[0.4658179281478898, "['Perceptron', 1, 2, True, True]"]
[0.45733054296012093, "['Perceptron', 1, 3, True, True]"]
[0.44884315777235206, "['Perceptron', 1, 1, True, True]"]
[0.44082083478665274, "['Perceptron multicouche', 1, 1, True, True]"]
[0.4337867689803511, "['Logistic Regression', 1, 2, True, True]"]
[0.395070340658063, "['Logistic Regression', 1, 3, True, True]"]
[0.38850133705383094, "['Perceptron multicouche', 1, 3, True, True]"]
[0.3855365655156377, "['Perceptron multicouche', 1, 2, True, True]"]
[0.35984187885129637, "['MultinomialNB', 1, 1, True, True]"]
[0.285257528194396, "['MultinomialNB', 1, 2, True, True]"]
[0.25863271712591557, "['MultinomialNB', 1, 3, True, True]"]
[0.16346936402743867, "['Decision Tree', 1, 3, True, True]"]
[0.15957

In [113]:
for res in sorted(Result,reverse=True):
    print(res)

[0.5264504127427043, "['linear_svc', 1, 1, True, True]"]
[0.5264504127427043, "['linear_svc', 1, 1, True, False]"]
[0.5264504127427043, "['linear_svc', 1, 1, False, True]"]
[0.5264504127427043, "['linear_svc', 1, 1, False, False]"]
[0.5227299151261481, "['linear_svc', 1, 2, True, True]"]
[0.5227299151261481, "['linear_svc', 1, 2, True, False]"]
[0.5227299151261481, "['linear_svc', 1, 2, False, True]"]
[0.5227299151261481, "['linear_svc', 1, 2, False, False]"]
[0.509010580165097, "['linear_svc', 1, 3, True, True]"]
[0.509010580165097, "['linear_svc', 1, 3, True, False]"]
[0.509010580165097, "['linear_svc', 1, 3, False, True]"]
[0.509010580165097, "['linear_svc', 1, 3, False, False]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, True, True]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, True, False]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, False, True]"]
[0.47942099755842343, "['Logistic Regression', 1, 1, False, False]"]
[0.4658179281478898, "['Perceptron',